In [ ]:
"""
Created on Thu Dec 07 10:42 2023

Trying to reproduce figures from Katherine

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'

In [5]:
T_grid_vars_closed = xr.open_dataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open = xr.open_dataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open = T_grid_vars_open.where(T_grid_vars_open['so'] > 0)

In [6]:
lon = T_grid_vars_open.nav_lon
lat = T_grid_vars_open.nav_lat

In [81]:
mask_Ant = lat < -50
mask_Weddell = (lon <= 5) & (lon >= -65.) & (lat >= -78.) & (lat <= -60.)
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)

In [11]:
# ANT
T_grid_vars_closed_Ant = T_grid_vars_closed.where(mask_Ant, drop=True)
T_grid_vars_open_Ant = T_grid_vars_open.where(mask_Ant, drop=True)
lon_Ant = T_grid_vars_open_Ant['nav_lon']
lat_Ant = T_grid_vars_open_Ant['nav_lat']

In [26]:
# Weddell
T_grid_vars_open_Wed = T_grid_vars_open.where(mask_Weddell, drop=True)
T_grid_vars_closed_Wed = T_grid_vars_closed.where(mask_Weddell, drop=True)
lon_Wed = T_grid_vars_open_Wed['nav_lon']
lat_Wed = T_grid_vars_open_Wed['nav_lat']

In [39]:
mask_Ross.plot()

In [82]:
# Ross
T_grid_vars_open_Ross = T_grid_vars_open.where(mask_Ross, drop=True)
T_grid_vars_closed_Ross = T_grid_vars_closed.where(mask_Ross, drop=True)
lon_Ross = T_grid_vars_open_Ross['nav_lon']
lat_Ross = T_grid_vars_open_Ross['nav_lat']

In [ ]:
T_grid_vars_closed_FRIS 

In [ ]:
T_grid_vars_open_FRIS['thetao'].isel(deptht=0).plot()

In [75]:
def maps_compare3(lon,lat,ref,modif,name,cmap,region,legend,lat_lim=-50):
    
    f = plt.figure(figsize=(8.25*3,8.25))
    #f = plt.figure()
    #f.suptitle(str(time_in.values)[0:16],fontsize=22)

    if region == 'global':
        proj = ccrs.Mollweide(central_longitude=0)
        wrap_ref = ref
        wrap_lon = lon
        wrap_modif = modif
        
    elif region == 'Ant':
        proj = ccrs.SouthPolarStereo(central_longitude=0)
        wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)
        wrap_modif, wrap_lon = modif, lon #add_cyclic_point(modif.values,coord=lon,axis=1)

        theta = np.linspace(0, 2*np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpl.path.Path(verts * radius + center)   

        extent = [-180, 180, -90, -50]
    
    elif region == 'Weddell':
        proj = ccrs.SouthPolarStereo(central_longitude=-32)
        wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)
        wrap_modif, wrap_lon = modif, lon #add_cyclic_point(modif.values,coord=lon,axis=1)

        extent = [-63, 0, -78, -64]
    
    elif region == 'Ross':
        cm = 180
        proj = ccrs.SouthPolarStereo(central_longitude=cm)
        wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)
        wrap_modif, wrap_lon = modif, lon #add_cyclic_point(modif.values,coord=lon,axis=1)
        minlon = -20 + cm
        maxlon = +50 + cm
        extent = [minlon, maxlon, -85, -70]
        
    #### REFERENCE
    ax1 = plt.subplot(1, 3, 1, projection=proj)
    abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),cmap=cmap,rasterized=True)
    ax1.coastlines(resolution='110m', linewidth=0.5)
    ax1.set_extent(extent, crs=ccrs.PlateCarree())

    if region == 'global':
        ax1.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    elif region == 'Ant':
        ax1.set_boundary(circle, transform=ax1.transAxes)

    if legend=='yes':
        cbar = f.colorbar(abso0, ax=ax1, shrink=0.3,orientation='vertical',extend='both')
        cbar.set_label(name,rotation=90)  

    #### MODIFICATION
    ax2 = plt.subplot(1, 3, 2, projection=proj)
    ax2.coastlines(resolution='110m', linewidth=0.5)
    abso = ax2.pcolormesh(wrap_lon,lat,wrap_modif,transform=ccrs.PlateCarree(),cmap=cmap,rasterized=True)
    ax2.set_extent(extent, crs=ccrs.PlateCarree())

    if region == 'global':
        ax2.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    elif region == 'Ant':
        ax2.set_boundary(circle, transform=ax1.transAxes)


    if legend=='yes':
        cbar = f.colorbar(abso0, ax=ax2, shrink=0.3,orientation='vertical',extend='both')
        cbar.set_label(name,rotation=90)    

    #### DIFFERENCE
    abs_lim = abs(modif - ref).quantile(0.99)
    ax3 = plt.subplot(1, 3, 3, projection=proj)
    ax3.coastlines(resolution='110m', linewidth=0.5)
    diff1 = ax3.pcolormesh(wrap_lon,lat,wrap_modif - wrap_ref,transform=ccrs.PlateCarree(),cmap=mpl.cm.coolwarm,rasterized=True,vmin=-abs_lim,vmax=abs_lim)
    ax3.set_extent(extent, crs=ccrs.PlateCarree())
    
    if region == 'global':
        ax3.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    elif region == 'Ant':
        ax3.set_boundary(circle, transform=ax1.transAxes)


    if legend=='yes':
        cbar = f.colorbar(diff1, ax=ax3, shrink=0.3,orientation='vertical',extend='both')
        cbar.set_label('Modif - ref '+name,rotation=90)    


    
    f.tight_layout()
    
    return f

SURFACE

In [76]:
#WEDDELL
f = maps_compare3(lon_Wed,lat_Wed,T_grid_vars_closed_Wed['thetao'].isel(deptht=0),T_grid_vars_open_Wed['thetao'].isel(deptht=0),'Temperature [°C]',cmocean.cm.thermal,'Weddell','yes')
f = maps_compare3(lon_Wed,lat_Wed,T_grid_vars_closed_Wed['so'].isel(deptht=0),T_grid_vars_open_Wed['so'].isel(deptht=0),'Salinity [g/kg]',cmocean.cm.haline,'Weddell','yes')

In [83]:
#ROSS
f = maps_compare3(lon_Ross,lat_Ross,T_grid_vars_closed_Ross['thetao'].isel(deptht=0),T_grid_vars_open_Ross['thetao'].isel(deptht=0),'Temperature [°C]',cmocean.cm.thermal,'Ross','yes')
f = maps_compare3(lon_Ross,lat_Ross,T_grid_vars_closed_Ross['so'].isel(deptht=0),T_grid_vars_open_Ross['so'].isel(deptht=0),'Salinity [g/kg]',cmocean.cm.haline,'Ross','yes')

BOTTOM

In [116]:
mask_cells = T_grid_vars_open['area'] > 0
bottom = mask_cells.sum('deptht')

In [130]:
T_grid_vars_closed_Wed['thetao']

<xarray.DataArray 'thetao' (deptht: 75, y: 54, x: 70)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [ 0.35547537,  0.19341543,  0.11205904, ..., -1.2428796 ,
         -1.2344167 , -1.2150127 ],
        [ 0.54870737,  0.39092726,  0.35010964, ..., -1.1928982 ,
         -1.1859062 , -1.1684394 ],
        [ 0.7937222 ,  0.6177735 ,  0.60855997, ..., -1.1659163 ,
         -1.1477691 , -1.1277936 ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]], dtype=float32)
Coordinates:
  * deptht   (deptht) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [131]:
bottom_Wed.plot()

In [141]:
#WEDDELL
bottom_Wed_nan = (bottom.where(mask_Weddell, drop=True) - 1)
bottom_Wed = (bottom_Wed_nan.where(np.isfinite(bottom_Wed_nan),0)).astype(int)

f = maps_compare3(lon_Wed,lat_Wed,T_grid_vars_closed_Wed['thetao'].isel(deptht=bottom_Wed),T_grid_vars_open_Wed['thetao'].isel(deptht=bottom_Wed),'Temperature [°C]',cmocean.cm.thermal,'Weddell','yes')
f = maps_compare3(lon_Wed,lat_Wed,T_grid_vars_closed_Wed['so'].isel(deptht=bottom_Wed),T_grid_vars_open_Wed['so'].isel(deptht=bottom_Wed),'Salinity [g/kg]',cmocean.cm.haline,'Weddell','yes')


/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
/home/cburgard/.conda/envs/py38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1556: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [144]:
#ROSS
bottom_Ross_nan = (bottom.where(mask_Ross, drop=True)  - 1)
bottom_Ross = (bottom_Ross_nan.where(np.isfinite(bottom_Ross_nan),0)).astype(int)

f = maps_compare3(lon_Ross,lat_Ross,T_grid_vars_closed_Ross['thetao'].isel(deptht=bottom_Ross),T_grid_vars_open_Ross['thetao'].isel(deptht=bottom_Ross),'Temperature [°C]',cmocean.cm.thermal,'Ross','yes')
f = maps_compare3(lon_Ross,lat_Ross,T_grid_vars_closed_Ross['so'].isel(deptht=bottom_Ross),T_grid_vars_open_Ross['so'].isel(deptht=bottom_Ross),'Salinity [g/kg]',cmocean.cm.haline,'Ross','yes')
